# Simple ML Jump (STUDENT VERSION)
Billy Hau - June 29, 2022

- Data Cleaning / Feature Engineering / Modeling 
- Deployment

In [1003]:
# Import Libraries
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import multiprocessing as mp

In [1004]:
# Read CSV File
df = pd.read_csv("./data.csv")
df


,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,-1.000000,-1.000000,-1.000000,-1.000000,False,True,True,False
1,38.196301,15.261410,-1.000000,-1.000000,False,True,True,False
2,34.373650,15.261410,-1.000000,-1.000000,False,True,True,False
3,30.305191,15.261410,41.488136,15.499226,False,True,True,False
4,26.230925,15.261410,37.350384,15.499226,False,True,True,False
...,...,...,...,...,...,...,...,...
6390,10.741413,16.315287,-1.000000,-1.000000,False,True,True,False
6391,6.656005,16.315287,-1.000000,-1.000000,False,True,True,False
6392,2.300749,16.315287,-1.000000,-1.000000,False,True,True,False
6393,-1.000000,-1.000000,-1.000000,-1.000000,False,True,True,False


In [1005]:
# What do each of the columns mean?

In [1006]:
jump = df[df['Jump'] == True]
dead = jump['Dead'].to_numpy()
# dead.value_counts()
for i in range(dead.shape[0]):
    #print(jump.index[i])
    try:
        if any(df['Dead'][jump.index[i] + x] for x in range(0, 11)):
            df.drop(df[df.index == jump.index[i]].index, inplace=True)
    except:
        pass
df

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,-1.000000,-1.000000,-1.000000,-1.000000,False,True,True,False
1,38.196301,15.261410,-1.000000,-1.000000,False,True,True,False
2,34.373650,15.261410,-1.000000,-1.000000,False,True,True,False
3,30.305191,15.261410,41.488136,15.499226,False,True,True,False
4,26.230925,15.261410,37.350384,15.499226,False,True,True,False
...,...,...,...,...,...,...,...,...
6390,10.741413,16.315287,-1.000000,-1.000000,False,True,True,False
6391,6.656005,16.315287,-1.000000,-1.000000,False,True,True,False
6392,2.300749,16.315287,-1.000000,-1.000000,False,True,True,False
6393,-1.000000,-1.000000,-1.000000,-1.000000,False,True,True,False


In [1007]:
dead = df['Dead'].to_numpy()
# dead.value_counts()
for i in range(df.shape[0]):
    # print(jump.index[i])
    try:
        if any(df['Dead'][df.index[i] + x] for x in range(0, 6)):
            print(df.index[i] + 5)
            df.drop(df[df.index == df.index[i]].index, inplace=True)
    except:
        pass
df


5
7
9
57
59
61
63
113
115
117
119
168
170
172
248
250
252
335
337
340
342
344
382
384
396
398
400
402
420
422
630
632
634
636
935
937
939
941
948
950
952
954
956
958
960
962
964
966
968
970
972
974
976
978
980
982
984
1001
1003
1005
1056
1058
1060
1118
1120
1122
1124
1126
1184
1186
1188
1805
1807
1809
1811
1813
1815
1817
1819
1821
1823
1825
1827
1829
1831
1833
2009
2011
2013
2015
2058
2060
2062
2122
2124
2126
2128
2130
2251
2253
2255
2257
2259
2278
2280
2282
2284
2286
2298
2300
2302
2304
2404
2406
2408
2410
2678
2680
2682
2684
2686
2688
2747
2749
2781
2783
2785
2787
2823
2825
2827
2829
2863
2865
2867
2952
2954
2956
2975
2977
2996
2998
3034
3036
3272
3274
3276
3278
3335
3337
3339
3466
3468
3470
3571
3573
3575
3577
3683
3685
3687
3689
3691
3697
3699
3701
3703
3705
3707
3713
3715
3717
3719
3721
3723
3725
3727
3874
3876
3878
3880
3882
3884
3913
3915
3917
3919
3994
3996
3999
4001
4003
4005
4007
4009
4095
4097
4145
4147
4149
4151
4172
4174
4176
4213
4215
4217
4219
4221
4223
4225
4246
4248
42

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
1,38.196301,15.261410,-1.000000,-1.000000,False,True,True,False
3,30.305191,15.261410,41.488136,15.499226,False,True,True,False
5,22.178299,15.261410,33.234604,15.499226,False,True,True,False
6,39.841648,16.293358,54.819611,16.459723,False,True,False,False
7,35.498672,16.293358,50.432281,16.459723,False,True,False,False
...,...,...,...,...,...,...,...,...
6385,16.462187,16.314571,31.416317,16.315287,False,True,False,False
6387,8.285727,16.314571,23.239498,16.315287,False,True,False,False
6389,0.147182,16.314571,15.100597,16.315287,False,True,True,False
6391,6.656005,16.315287,-1.000000,-1.000000,False,True,True,False


In [1008]:
# Are there data we don't care about?
df.drop(df[df['Dead'] == True].index, inplace=True)
df.drop(df[df['Grounded'] == False].index, inplace=True)
df = df.drop(["Grounded", "Dead", "Scored"], inplace=False, axis=1)
df

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump
6,39.841648,16.293358,54.819611,16.459723,False
7,35.498672,16.293358,50.432281,16.459723,False
8,31.420794,16.293358,46.312759,16.459723,False
9,27.081730,16.293358,41.929390,16.459723,False
10,22.999084,16.293358,37.805058,16.459723,False
...,...,...,...,...,...
6381,33.310596,16.314571,-1.000000,-1.000000,False
6382,29.222736,16.314571,-1.000000,-1.000000,False
6383,24.885532,16.314571,39.840034,16.315287,False
6385,16.462187,16.314571,31.416317,16.315287,False


In [1009]:
jumpCount = df[df['Jump'] == True].shape[0]
jumpCount

554

In [1010]:
noJumpCount = df[df['Jump'] == False].shape[0]
noJumpCount

3115

In [1011]:
df_toDrop = df[df['Jump'] == False].sample(noJumpCount - jumpCount)
df.drop(df_toDrop.index, inplace=True)

In [1012]:
# Create the Input DataFrame
X = df[['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance', 'Bar 2 Speed']]
X = X.to_numpy()

In [1013]:
# Create the Output DataFrame and Map True / False to 1 / 0
Y = df[['Jump']]


def bool2Num(row):
    if row['Jump']:
        return 1
    else:
        return 0


Y['Jump'] = Y.apply(bool2Num, axis='columns')
Y = Y['Jump'].to_numpy()

/tmp/ipykernel_1114684/2096279593.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Jump'] = Y.apply(bool2Num, axis='columns')


In [1014]:
# Train Test Split with test size set to 20%
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [1015]:
from sklearn.linear_model import LogisticRegression

# Define and Build Model (Logistic Regression)
model = RandomForestClassifier()
model.fit(x_train, y_train)


RandomForestClassifier()

In [1016]:
predict = model.predict(x_test)

In [1017]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [1018]:
# Print Out Prediction Accuraccy
accuracy_score(y_test, predict)

0.9480874316939891

In [1019]:
# Try a Different Model (Random Forest)
"""model2 = ________________________________
model2.__________________________

predict = ___________________________
accuracy_score(_________________, ________________)"""


'model2 = ________________________________\nmodel2.__________________________\n\npredict = ___________________________\naccuracy_score(_________________, ________________)'

# Deployment

In [1020]:
# Command to Open SIMPLE ML Jump 2 (for Example)

# Windows OS
#env_path = "D:\\User\\Desktop\\10botics Data Science\\ML Game\\SimpleMLJump2 Builds\\Windows\\Simple ML Jump 2.exe"

# MacOS
#env_path = 'open -n "/Users/billwaa/Desktop/ML Game/Simple ML Jump 2.app"' 

env_path = "/home/wcyat/dev/kaggle/code/ml-jump/Linux/SimpleMLJump2.x86_64"

In [1021]:
# Import Libraries
import socket
import struct
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import time
import subprocess
import platform
import collect
import multiprocessing as mp


# Find Open Socket Ports
def findOpenSockets(size: int):
    sock = []
    port = []

    for i in range(size):
        sock.append(socket.socket())
        sock[i].bind(('localhost', 0))
        port.append(sock[i].getsockname()[1])

    for i in range(len(sock)):
        sock[i].close()

    return port


# Find Open Ports and Launch Game Environment
envNum = 1
openPorts = findOpenSockets(envNum * 2)
gamePort = openPorts[0]
apiPort = openPorts[1]

# Determine OS and Launch 
if platform.system() == 'Darwin':
    env_path += f" --args --apiPort {str(apiPort)} --gamePort {str(gamePort)} --small false"
    process = subprocess.Popen([env_path], shell=True)
else:
    process = subprocess.Popen([env_path, '--apiPort', str(apiPort), '--gamePort', str(gamePort), '--small', 'false'])

# Establish UDP Network Client
localIP = "localhost"
bufferSize = 1024
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
UDPServerSocket.bind((localIP, apiPort))

# Set UDP Timeout
UDPServerSocket.settimeout(3)

# Wait for Game Environment to Open
t0 = time.time()

while (time.time() - t0 < 5):
    pass


# Extract Data
def extractData(data):
    dist1 = struct.unpack('f', data[:4])
    speed1 = struct.unpack('f', data[4:8])
    dist2 = struct.unpack('f', data[8:12])
    speed2 = struct.unpack('f', data[12:16])

    onGround = True if (data[16] & (1 << 0)) == 1 else False
    toJump = True if (data[17] & (1 << 0)) == 1 else False
    isDead = True if (data[18] & (1 << 0)) == 1 else False
    scored = True if (data[19] & (1 << 0)) == 1 else False

    dat = [dist1[0], speed1[0], dist2[0], speed2[0], toJump, onGround, isDead,
           scored]  # Swap Order to Accomadate Old Model
    df_temp = pd.DataFrame(dat).transpose()

    return df_temp


# Reset Simulation
UDPServerSocket.sendto(bytes.fromhex('07 00'), (localIP, gamePort))

2

In [1022]:
def autoJump():
# Process Loop - Retrieve Data from Simulation, Run Through Model, Output Command
    while True:
        try:

            bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)

            t0 = time.time()

            message = bytesAddressPair[0]
            address = bytesAddressPair[1]
            df_temp = extractData(message)
            X = df_temp.iloc[:, :4]
            X = X.astype(float)

        # Data PreProcessing Function Go Here

        # Model Prediction Function Go Here
            predict = model.predict(X)

            print(predict, end='\r')

        # If Jump Decision is Made, Send to Game
            if predict[0] > 0.5:
            # Jump Command
                UDPServerSocket.sendto(bytes.fromhex('06 01'), (localIP, gamePort))


        except Exception as e:
            print(e)
            break

In [1023]:
autoJump()

timed out


In [1024]:
# Close UDP Port When Game is Closed
UDPServerSocket.close()